In [ ]:
import os 
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate , SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA

#agents (I havent used it coz it was never needed)
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

#for langserve
from fastapi import FastAPI
import uvicorn

In [25]:
#loader instance
loader = CSVLoader('ipc_sections.csv',encoding="utf-8")

#loading the documents from Csv file
doc = loader.load()

In [26]:
#splitting the data 
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.split_documents(doc)

In [27]:
 #embedding the data
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
#faiss db
#Storing into Vector Data Base

db_faiss = "faiss_db"
db = FAISS.from_documents(chunks, embeddings)
db.save_local(db_faiss)
print("Saved into the vector database")

Saved into the vector database


In [28]:
# Loading a LLm 
GROQ_API_KEY = os.getenv("GROQ_API")

llm = ChatGroq(
   model_name="llama3-8b-8192",
   api_key= GROQ_API_KEY)

In [29]:
# Creating the ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "As a legal assistant specializing in criminal law in the Indian Penal Code (IPC), your task is to identify applicable legal sections relevant to the provided scenario. Below is the user's query. List all applicable legal sections for the scenario. If the provided context does not contain sufficient information to determine the relevant sections, respond with 'I'm unable to provide an answer based on the available information.'"
    ),
    HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nQuery: {question}"
    )
])

In [30]:
#loading the stored vector DB
db_faiss = "faiss_db"
db = FAISS.load_local(db_faiss,
                      embeddings,
                      allow_dangerous_deserialization=True)

In [39]:
# Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 30}),
                                       return_source_documents=True,
                                       verbose = True,
                                       chain_type_kwargs={'prompt': chat_prompt_template}
                                        )

In [40]:
# Query the model
question = input("Enter your question: ")
result = qa_chain({"query": question})

# Extract and print the answer
answer = result["result"]
print("\nAnswer:", answer)



> Entering new RetrievalQA chain...

> Finished chain.

Answer: Based on the provided context and applicable legal sections, I have identified the following sections relevant to the scenario of married women:

1. IPC 494 - Marrying again during the life-time of a husband or wife (punishment: 7 Years + Fine)
2. IPC 498 - Enticing or taking away or detaining with a criminal intent a married woman (punishment: 2 Years and Fine)
3. IPC 304B - Dowry death (punishment: Imprisonment for not less than 7 Years, but upto Life)
4. IPC 376B - Sexual intercourse by husband upon his wife during separation (punishment: 2 to 7 years + Fine)
5. IPC 498A - Punishment for subjecting a married woman to cruelty (punishment: 3 Years + Fine)
6. IPC 376C - Sexual intercourse by a person in authority (punishment: Rigorous Imprisonment for 5 to 10 years + Fine)
7. IPC 497 - Adultery (punishment: 5 Years or Fine or Both)
8. IPC 493 - A man by deceit causing a woman not lawfully married to him to believe that s